In [2]:
import bs4
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import re
import requests 


In [1]:
import googleapiclient.discovery
import google_auth_oauthlib.flow
from keras.models import load_model
import os 
os.chdir('..\data')
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
CLIENT_SECRET_FILE = 'client_secret2.json'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly', 'https://www.googleapis.com/auth/youtube.force-ssl']

# API 클라이언트 생성 및 인증하여 Credential 객체 얻기
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=29898966518-ra5akcd4ltrohgh0igfsi5dkmc87g4a7.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=KKI28shoK9JkfOBZExNT7l9CK7d0rR&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g7b6KzIyYGmC3LzwVCx8GEKzYc0nLxCyicuIifQIvtxORhfjIeJ4oc


In [21]:
link = 'https://www.youtube.com/results?search_query='
my_driver = webdriver.Chrome('chromedriver.exe') 
my_driver.get(link+"검색어 입력")
my_html = my_driver.page_source  
my_driver.close()

In [22]:
my_soup = bs4.BeautifulSoup(my_html, 'html.parser')    

video_id = my_soup.select("div > h3 > a")

In [23]:
r = re.compile("/watch\?v\=")
r1= re.compile("\W")

In [24]:
video_id_list = []
for v in video_id[:2]:
    video_id_list.append([r.sub("",str(v["href"])),r1.sub("",v.text)])

In [25]:
video_id_list

[['x53fuemtmsQ', '14화몰아보기섹시하게일하고애틋하게사랑하라검블유검색어를입력하세요WWWSearchWWW'],
 ['Tf89th_UWrY', 'tvN유튜브멤버십OPEN검색어를입력하세요WWW1화10분미리보기']]

In [27]:
import pandas
from googleapiclient.discovery import build
 
 

for i,(v_id,v_name) in enumerate(video_id_list):
    try:
        video_id = v_id
        
        comments = list()
#         api_obj = build('youtube', 'v3', developerKey=api_key)
        response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute()
        
        while response:
            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']
                comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
                
                if item['snippet']['totalReplyCount'] > 0:
                    for reply_item in item['replies']['comments']:
                        reply = reply_item['snippet']
                        comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
            if 'nextPageToken' in response:
                response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
            
            
            else:
                break
    except:
        print("오류")
        continue
 
    df = pandas.DataFrame(comments)
    df.to_excel(v_name+".xlsx", header=['comment', 'author', 'date', 'num_likes'], index=None)

In [33]:
response = youtube.commentThreads().list(part='snippet,replies', videoId='x53fuemtmsQ', maxResults=500).execute()

100